In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<style>
input {
    background-color: #3c83bb;
    color: white;
    border: none;
    font-family: sans-serif;
    padding: 8px;
    border-radius: 10px;
    
}
</style>
<form action="javascript:code_toggle()"><input type="submit" value="See Raw Code"></form>''')


In [2]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import display, clear_output
from jupyterthemes import jtplot

jtplot.style(theme='chesterish', grid=False)


In [3]:
df_prod_factors = pd.read_csv("../Data/Productivity_Factors.csv", index_col=0)
df_prod_grades_alpha = pd.read_csv("../Data/Productivity_Grades_Alpha.csv", index_col=0)
df_prod_grades_num = pd.read_csv("../Data/Productivity_Grades_Num.csv", index_col=0)
df_prof_factors = pd.read_csv("../Data/Profitability_Factors.csv", index_col=0)
df_prof_grades_alpha = pd.read_csv("../Data/Profitability_Grades_Alpha.csv", index_col=0, dtype={'High GMR': object, 'High Sales and GM Weight': object, 'Equal Weights': object})
df_prof_grades_num = pd.read_csv("../Data/Profitability_Grades_Num.csv", index_col=0)


In [4]:
df_display_prod = df_prod_grades_num[['Sales District Name', 'Sales Office', 'Date (YYYYMM)', 'High Sales Weight', 'Equal Sales and GM Weight', 'Equal Weights']].copy()
df_display_prod[['High Sales Weight Alpha', 'Equal Sales and GM Weight Alpha', 'Equal Weights Alpha']] = df_prod_grades_alpha[['High Sales Weight', 'Equal Sales and GM Weight', 'Equal Weights']].copy()

df_display_prof = df_prof_grades_num[['Sales District Name', 'Sales Office', 'Date (YYYYMM)', 'High GMR', 'High Sales and GM Weight', 'Equal Weights']].copy()
df_display_prof[['High GMR Alpha', 'High Sales and GM Weight Alpha', 'Equal Weights Alpha']] = (df_prof_grades_alpha[['High GMR', 'High Sales and GM Weight', 'Equal Weights']].copy())

df_display_prof.replace("nan", np.nan, inplace=True)

dates = ['201701', '201702', '201703', '201704', '201705', '201706', '201707', 
         '201708', '201709', '201710', '201711', '201712', '201801', '201802', 
         '201803', '201804', '201805', '201806', '201807', '201808', '201809', 
         '201810', '201811', '201812', '201901', '201902', '201903']

branches = df_display_prod['Sales Office'].unique()
district = df_display_prod['Sales District Name'].unique()

district_branch = dict(zip(df_display_prod['Sales Office'], df_display_prod['Sales District Name']))

In [5]:
Productivity = widgets.Dropdown(
    options=['Equal Weights', 'High Sales Weight', 'Equal Sales and GM Weight'],
    value='Equal Weights',
    description='Productivity',
)

Profitability = widgets.Dropdown(
    options=['Equal Weights', 'High GMR', 'High Sales and GM Weight'],
    value='Equal Weights',
    description='Profitability',
)

timeline = widgets.Dropdown(
    options=dates,
    value='201701',
    description='Timeline',
)

Value_Options = widgets.Dropdown(
    options=['Productivty & Profitability','Productivity', 'Profitability'],
    value='Productivty & Profitability',
    description='Values',
)

Branch = widgets.Dropdown(
       options=['ALNY','ALPA','HAPA','HVNY','NCDE','PHPA','TENJ','WIPA'],
       value='ALNY',
       description='Branch',
)

District = widgets.Dropdown(
   options=district,
   value='New York',
   description='District',
)

Within_District = widgets.Dropdown(
    options=['True', 'False'],
    value='True',
    description='Within District',
)

Prod_text = widgets.HTML(
    value="<b><h4><u>Productivity Scores</h4></u></b>",
)
Prof_text = widgets.HTML(
    value="<b><h4><u>Profitability Scores</h4></u></b>",
)

Submit = widgets.Button(
    description='Submit',
    button_style='success',
    tooltip='Submit',
    icon='check'
)

def update(*args):
    filtered_list=[]
    for k,v in district_branch.items():
        if v==District.value:
            filtered_list.append(k)
    Branch.options=filtered_list

def on_button_clicked(Submit):
    with out:
        clear_output()
        
        prod_columns = ['Sales District Name', 'Sales Office', 'Date (YYYYMM)',
                        'GM per Emp', 'Sales Order per Emp', 'Sales Order Items per Emp', 
                        'Sales Order Delivery per Emp','Non-Sales Order Delivery per Emp', 
                        'Delivery Lines per Emp', 'Deliver Docs per Emp']
        prof_columns = ['Sales District Name', 'Sales Office', 'Date (YYYYMM)',
                        'Number of Delivery Lines (SO)','Sales $ per branch',
                        'GM $ per branch', 'RSO by plant','GMR','Sales Order - RSO']
        
        temp_df_prod_factors = df_prod_factors.copy()
        temp_df_prod_factors[Productivity.value + ' vs ' + Profitability.value] = df_prod_grades_alpha[Productivity.value] + df_prof_grades_alpha[Profitability.value]
        
        temp_df_prof_factors = df_prof_factors.copy()
        temp_df_prof_factors[Productivity.value + ' vs ' + Profitability.value] = df_prod_grades_alpha[Productivity.value] + df_prof_grades_alpha[Profitability.value]
        
        productivity_row = df_prod_grades_alpha[(df_prod_grades_alpha['Sales Office'] == Branch.value) & (df_prod_grades_alpha['Date (YYYYMM)'] == int(timeline.value))]
        productivity_score = productivity_row.iloc[0].loc[Productivity.value]
        district_value_prod =  productivity_row.iloc[0].loc['Sales District']
        profitability_row = df_prof_grades_alpha[(df_prof_grades_alpha['Sales Office'] == Branch.value) & (df_prof_grades_alpha['Date (YYYYMM)'] == int(timeline.value))]
        profitability_score = profitability_row.iloc[0].loc[Profitability.value]
        district_value_prof =  profitability_row.iloc[0].loc['Sales District']
        
        if Value_Options.value == 'Productivity':
            Productivity_row = temp_df_prod_factors[(temp_df_prod_factors[Productivity.value + ' vs ' + Profitability.value]==productivity_score+profitability_score) & (temp_df_prod_factors['Date (YYYYMM)'] == int(timeline.value))]
            if Within_District.value == 'True':
                District_Productivity_row = Productivity_row[Productivity_row['Sales District']==district_value_prod]
                display(Prod_text, District_Productivity_row[prod_columns])
            else:
                display(Prod_text, Productivity_row[prod_columns])
        elif Value_Options.value == 'Profitability':
            Profitability_row = temp_df_prof_factors[(temp_df_prof_factors[Productivity.value + ' vs ' + Profitability.value]==productivity_score+profitability_score) & (temp_df_prod_factors['Date (YYYYMM)'] == int(timeline.value))]
            if District.value == 'True':
                District_Profitability_row = Profitability_row[Profitability_row['Sales District']==district_value_prof]
                display(Prof_text, District_Profitability_row[prof_columns])
            else:
                display(Prof_text, Profitability_row[prof_columns])
        else:
            if Within_District.value == 'True':
                Productivity_row = temp_df_prod_factors[(temp_df_prod_factors[Productivity.value + ' vs ' + Profitability.value]==productivity_score+profitability_score) & (temp_df_prod_factors['Date (YYYYMM)'] == int(timeline.value))]
                District_Productivity_row = Productivity_row[Productivity_row['Sales District']==district_value_prod]
                Profitability_row = temp_df_prof_factors[(temp_df_prof_factors[Productivity.value + ' vs ' + Profitability.value]==productivity_score+profitability_score) & (temp_df_prod_factors['Date (YYYYMM)'] == int(timeline.value))]
                District_Profitability_row = Profitability_row[Profitability_row['Sales District']==district_value_prof]
                display(Prod_text, District_Productivity_row[prod_columns],Prof_text,  District_Profitability_row[prof_columns])
                
            else:
                Productivity_row = temp_df_prod_factors[(temp_df_prod_factors[Productivity.value + ' vs ' + Profitability.value]==productivity_score+profitability_score) & (temp_df_prod_factors['Date (YYYYMM)'] == int(timeline.value))]
                Profitability_row = temp_df_prof_factors[(temp_df_prof_factors[Productivity.value + ' vs ' + Profitability.value]==productivity_score+profitability_score) & (temp_df_prod_factors['Date (YYYYMM)'] == int(timeline.value))]
                display(Prod_text, Productivity_row[prod_columns],Prof_text,  Profitability_row[prof_columns])

Submit.on_click(on_button_clicked)
District.observe(update)

hbox1 = widgets.HBox([Productivity, Profitability, Value_Options])
hbox2 = widgets.HBox([timeline, District, Within_District])

box_layout = widgets.Layout(display='flex',
                flex_flow='column',
                align_items='center',
                width='100%')

vbox = widgets.VBox([hbox1, hbox2, Branch, Submit], layout=box_layout)


In [6]:
out = widgets.Output()
display(out)


display(vbox)

Output()

<center>
    <img src=../Legend/Productivity_and_Profitability_Graphs.png  width="500000" height="250000" />
</center